In [1]:
import llama_index
from dotenv import load_dotenv
import os

In [2]:
load_dotenv()

OPENAI_API_KEY=os.getenv('OPENAI_API_KEY')

In [3]:
#Bring in dependencies

from llama_index.core.agent import ReActAgent
from llama_index.llms.openai import OpenAI
from llama_index.core.tools import FunctionTool

# Create basic tools

In [4]:
def multiply(a: float,b: float)-> float:
    """Multiply two numbers and return the product"""
    return a*b

multiply_tool=FunctionTool.from_defaults(fn=multiply)

def add(a: float,b: float)-> float:
    """Add two numbers and return the sum"""
    return a+b

add_tool=FunctionTool.from_defaults(fn=add)


As you can see, these are regular vanilla Python functions. The docstring comments provide metadata to the agent about what the tool does: if your LLM is having trouble figuring out which tool to use, these docstrings are what you should tweak first.

After each function is defined we create FunctionTool objects from these functions, which wrap them in a way that the agent can understand.

# Initialize the LLM

In [5]:
llm = OpenAI(model="gpt-3.5-turbo", temperature=0)

# Initialize the agent

In [6]:
agent = ReActAgent.from_tools([multiply_tool, add_tool], llm=llm, verbose=True)

In [30]:
response = agent.chat("What is 20+(2*4)? Use a tool to calculate every step.")

> Running step e121913b-29a9-4b8b-b2e9-53637027f9f9. Step input: What is 20+(2*4)? Use a tool to calculate every step.
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: add
Action Input: {'a': 20, 'b': 8}
Observation: 28
> Running step 68246538-80ef-4c75-800a-0a474b8d4058. Step input: None
Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: 20 + (2 * 4) = 20 + 8 = 28


# Switch to local agent

In [31]:
#!pip install llama-index-llms-ollama

In [32]:
#from llama_index.llms.ollama import Ollama

In [34]:
# llm = Ollama(model="mixtral:8x7b", request_timeout=120.0)
# agent = ReActAgent.from_tools([multiply_tool, add_tool], llm=llm, verbose=True)
# response = agent.chat("What is 20+(2*4)? Use a tool to calculate every step.")

# Adding RAG to an agent

In [7]:
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, Settings

In [8]:
Settings.llm = OpenAI(model="gpt-3.5-turbo", temperature=0)

In [9]:
documents = SimpleDirectoryReader("./data").load_data()
index = VectorStoreIndex.from_documents(documents)
query_engine = index.as_query_engine()

# Add a query engine tool

In [10]:
from llama_index.core.tools import QueryEngineTool

In [46]:
paper_tool=QueryEngineTool.from_defaults(
    query_engine,
    name='attention all you need',
    description='A RAG engine with advance knowledge of attention all you need'
)

In [47]:
agent=ReActAgent.from_tools([multiply_tool,add_tool,paper_tool],verbose=True)

response=agent.chat('What is attention ? use paper_tool for query')

print(response)

> Running step 2c6dcaf5-4d63-4f69-a44a-44c3bf71222b. Step input: What is attention ? use paper_tool for query
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: attention
Action Input: {'input': 'What is attention?'}
Observation: Error: No such tool named `attention`.
> Running step f895d2ef-ebd3-40d6-9361-ca488add9077. Step input: None
Thought: I cannot answer the question with the provided tools.
Answer: I cannot answer the question with the provided tools.
I cannot answer the question with the provided tools.


# Advance tools

In [11]:
!pip install llama-index-tools-yahoo-finance

  Using cached llama_index_tools_yahoo_finance-0.1.1-py3-none-any.whl.metadata (1.8 kB)
  Using cached yfinance-0.2.41-py2.py3-none-any.whl.metadata (11 kB)
  Using cached multitasking-0.0.11-py3-none-any.whl.metadata (5.5 kB)
  Using cached peewee-3.17.6-cp311-cp311-linux_x86_64.whl
  Using cached html5lib-1.1-py2.py3-none-any.whl.metadata (16 kB)
Using cached llama_index_tools_yahoo_finance-0.1.1-py3-none-any.whl (2.7 kB)
Using cached yfinance-0.2.41-py2.py3-none-any.whl (73 kB)
Using cached html5lib-1.1-py2.py3-none-any.whl (112 kB)
Using cached multitasking-0.0.11-py3-none-any.whl (8.5 kB)


In [12]:
from llama_index.tools.yahoo_finance import YahooFinanceToolSpec

In [13]:
finance_tools=YahooFinanceToolSpec().to_tool_list()

In [18]:
finance_tools.extend([add_tool,multiply_tool])

In [19]:
agent=ReActAgent.from_tools(finance_tools,verbose=True)

response=agent.chat('What is the current price of NVDA ?')

print(response)

> Running step 7c2babfa-2be9-4ab5-9f8c-9bcc9d024bec. Step input: What is the current price of NVDA ?


AttributeError: 'function' object has no attribute 'metadata'